In [1]:
import os
import numpy as np
import pandas as pd
import warnings
import func
from multiprocessing import Pool
from tqdm import tqdm
from params import dresden_images_root, train_csv_path, patch_span, \
        patch_num, patches_root, patches_db_path, weights_path

In [2]:
warnings.filterwarnings("ignore")

if not os.path.exists(patches_root):
    os.makedirs(patches_root)

In [3]:
images_db = pd.read_csv(train_csv_path)

model_list = np.unique(images_db['brand_model'])
img_list = images_db['path']

if not os.path.exists(patches_db_path):
    train_list, val_list, test_list = func.split(img_list, model_list, patches_db_path)
else:
    patches_db = np.load(patches_db_path, allow_pickle=True).item()
    train_list = patches_db['train']
    val_list = patches_db['val']
    test_list = patches_db['test']
    info, weights = func.split_info(train_list, val_list, test_list, 
                                       model_list, total=len(img_list))

# store class weight to csv
df = pd.DataFrame([weights], columns=range(len(model_list)))
df.to_csv(weights_path, index=False)

Canon_Ixus70 in training set: 352.
Canon_Ixus70 in validation set: 103.
Canon_Ixus70 in test set: 112.

Nikon_D200 in training set: 490.
Nikon_D200 in validation set: 113.
Nikon_D200 in test set: 149.

Olympus_mju-1050SW in training set: 669.
Olympus_mju-1050SW in validation set: 161.
Olympus_mju-1050SW in test set: 210.



In [4]:
for i in range(len(model_list)):
    func.patch(model_list[i], info[i][0], 'train')
    func.patch(model_list[i], info[i][1], 'val')
    func.patch(model_list[i], info[i][2], 'test')

100%|██████████| 210/210 [00:00<00:00, 329889.08it/s]
